## Segmenting and Clustering Neighborhoods in Toronto ##

### Part 3 - Applied Data Science Capstone Project ###

In this part we Explore and cluster the neighborhoods in Toronto.
This code includes Part 1 and Part 2 in order to keep it self-contained.

In [36]:
# Import main libraries for this project.

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [2]:
# First extract the url using request and create a BeautifulSoup object.

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
soup = BeautifulSoup(extracting_data, "lxml")


In [3]:
# Now extract the table and process its information.

table_contents=[]
table=soup.find("table")

for row in table.findAll("td"):    
    cell = {}
    
    if row.span.text == "Not assigned":
        pass
    
    else:
        cell["PostalCode"] = row.p.text[:3]
        cell["Borough"] = (row.span.text).split("(")[0]
        cell["Neighborhood"] = (((((row.span.text).split("(")[1]).strip(")")).replace(" /", ",")).replace(")"," ")).strip(" ")
        table_contents.append(cell)
        

In [4]:
# Convert the data to a Pandas DataFrame.

df = pd.DataFrame(table_contents)
df["Borough"] = df["Borough"].replace({"Downtown TorontoStn A PO Boxes25 The Esplanade":"Downtown Toronto Stn A",
                                        "East TorontoBusiness reply mail Processing Centre969 Eastern":"East Toronto Business",
                                        "EtobicokeNorthwest":"Etobicoke Northwest",
                                        "East YorkEast Toronto":"East York/East Toronto",
                                        "MississaugaCanada Post Gateway Processing Centre":"Mississauga"})


In [5]:
# Do some data processing.

df = df[df["Borough"] != "Not assigned"]
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()


In [6]:
# Print the DataFrame.

df


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
# Shape of the DataFrame.

print(df.shape)


(103, 3)


### From here starts Part 2 of the Project ###

In [8]:
# Install and import geocoder package.

!pip install geocoder    # uncomment if you need to install the package.
import geocoder


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [9]:
# Define function to get the latitude of longitude given a Postal Code.

def get_geocoords(postal_code):
    
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google("{}, Toronto, Ontario".format(postal_code))
        lat_lng_coords = g.latlng

    return lat_lng_coords


#### Currently geocoder service is really slow, switch to csv file to rescue geospatial information ####

In [10]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Add corresponding columns to DataFrame

df = pd.merge(df, df_geo, how="left", left_on="PostalCode", right_on="Postal Code").drop("Postal Code", axis=1)


In [12]:
# Print resulting DataFrame.

df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


### From here starts Part 3 of the Project ###

In [13]:
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim
!pip install folium
import folium # map rendering library

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 6.1 MB/s  eta 0:00:01


In order to define a geocoder instance we need an user_agent. Once created we define a geocoder instance for Toronto and draw a map.

In [14]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geograpical coordinate of Toronto are {}, {}.".format(latitude, longitude))


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
# Create a map of Toronto using latitude and longitude values.

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map.

for lat, lng, borough, neighborhood in zip(df["Latitude"], df["Longitude"], df["Borough"], df["Neighborhood"]):
    label = "{}, {}".format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color="blue",
        fill=True,
        fill_color="#3186cc",
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    

In [16]:
# Draw the map.

map_toronto

Following the project suggestions, let's simplify the above map and segment and cluster only the neighborhoods whose name contains Toronto. Once sliced the original DataFrame create a map with the subset.

In [17]:
df_toronto = df[df["Borough"].str.contains("Toronto")].reset_index(drop=True)

df_toronto.head()


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


In [18]:
# create map of Toronto using latitude and longitude values. Only mark neighborhoods containing the word Toronto.

map_subtoronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map

for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_subtoronto)  
    
map_subtoronto

#### Now we are going to start to use Foursquare API to explore the neighborhoods and segment them.

In [19]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: DUBP4QPTEZHLUFIXQINXG5SPF4UPCN3K2BKVNMLEJVVOUGS2
CLIENT_SECRET:ZJ1KQ23BFUAS5WYODKGUTDS215UPRTWCLXAYRKCOZEDEZ0RM


In [20]:
# This funcion gets nearby venues to locations given by its geospatial coordinates.

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:
# Now create a DataFrame containing nearby venues to each neighborhood in Toronto.

toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                 latitudes=df_toronto['Latitude'],
                                 longitudes=df_toronto['Longitude'])


The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [22]:
# Check the size of the resulting DataFrame.

print(toronto_venues.shape)
toronto_venues.head()


(1577, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [23]:
# Number of venues per Neighborhood.

toronto_venues.groupby('Neighborhood')["Venue"].count()


Neighborhood
Berczy Park                                                                                                    59
Brockton, Parkdale Village, Exhibition Place                                                                   23
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     16
Central Bay Street                                                                                             60
Christie                                                                                                       16
Church and Wellesley                                                                                           78
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                                                     34
Davisville North                                                           

In [24]:
# Check unique categories from returned values.

print("There are {} uniques categories.".format(len(toronto_venues["Venue Category"].unique())))


There are 232 uniques categories.


### Neighborhood Analysis

In [25]:
# one hot encoding of Venue Category.

toronto_onehot = pd.get_dummies(toronto_venues[["Venue Category"]], prefix="", prefix_sep="")

# add neighborhood column back to dataframe

toronto_onehot["Neighborhood"] = toronto_venues["Neighborhood"] 

# move neighborhood column to the first column

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
# Groupby Neighborhood and get mean of occurrences of each Venue Category.

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped


,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016949,0.00000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
3,Central Bay Street,0.016667,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.016667,0.00000,0.000000,0.016667,0.000000
4,Christie,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.025641,0.012821,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012821,...,0.012821,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.00000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.029412,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000


#### From above data we will create a DataFrame with 10 most frequent venue categories per Neighborhood ####

In [27]:
# First create a function that sorts the venue categories in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [32]:
# Now get the 10 most frequent venue categories per neighborhood and save the result in a DataFrame.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Beer Bar,Seafood Restaurant,Cheese Shop,Pharmacy,Restaurant,Jazz Club
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Bakery,Coffee Shop,Furniture / Home Store,Intersection,Restaurant,Italian Restaurant,Burrito Place,Stadium
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Coffee Shop,Rental Car Location,Harbor / Marina,Boat or Ferry,Bar,Airport Gate,Airport Food Court
3,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Poke Place,Park
4,Christie,Grocery Store,Café,Park,Athletics & Sports,Nightclub,Restaurant,Candy Store,Italian Restaurant,Baby Store,Coffee Shop


### Neighborhood Clustering

In [33]:
# set number of clusters

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop("Neighborhood", 1)

# run k-means clustering

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe

kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [34]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, "Cluster Labels", kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index("Neighborhood"), on="Neighborhood")

toronto_merged.head() # check the last columns!


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Asian Restaurant,Health Food Store,Pub,Trail,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Wine Shop
1,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106,2,Park,Convenience Store,Wine Shop,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge,Liquor Store
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Sushi Restaurant,Food & Drink Shop,Brewery,Board Shop,Sandwich Place,Italian Restaurant,Restaurant,Ice Cream Shop
4,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Gastropub,Bakery,Brewery,Café,American Restaurant,Yoga Studio,Comfort Food Restaurant,Bookstore,Seafood Restaurant


Finally, create a map with the resulting clusters.

In [37]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged["Latitude"], toronto_merged["Longitude"], toronto_merged["Neighborhood"], toronto_merged["Cluster Labels"]):
    label = folium.Popup(str(poi) + " Cluster " + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters